<a href="https://colab.research.google.com/github/naikhpratik/Aircraft-Simulation/blob/master/nb/Advanced_Llama3_1_(3B)_GRPO_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [4]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [5]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

In [ ]:
import os
os.kill(os.getpid(), 9)

In [6]:
import torch
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

Load up `Llama 3.1 8B Instruct`, and set parameters

In [7]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.8, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3. vLLM: 0.8.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Llama-3.2-3B-Instruct with actual GPU utilization = 79.24%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 5.52 GB. Also swap space = 2 GB.
WARNING 04-08 15:06:53 [config.py:2704] Casting torch.bfloat16 to torch.float16.
INFO 04-08 15:07:18 [config.py:600] This model supports multiple tasks: {'score', 'generate', 'embed', 'reward', 'classify'}.

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 04-08 15:08:05 [loader.py:447] Loading weights took 41.31 seconds
INFO 04-08 15:08:05 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 04-08 15:08:05 [model_runner.py:1146] Model loading took 6.2472 GiB and 42.418708 seconds
INFO 04-08 15:08:12 [worker.py:267] Memory profiling takes 5.96 seconds
INFO 04-08 15:08:12 [worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.79) = 11.68GiB
INFO 04-08 15:08:12 [worker.py:267] model weights take 6.25GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.89GiB; the rest of the memory reserved for KV Cache is 4.51GiB.
INFO 04-08 15:08:12 [executor_base.py:112] # cuda blocks: 2640, # CPU blocks: 1170
INFO 04-08 15:08:12 [executor_base.py:117] Maximum concurrency for 1024 tokens per request: 41.25x
INFO 04-08 15:08:15 [model_runner.py:1456] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager

Capturing CUDA graph shapes: 100%|██████████| 27/27 [00:35<00:00,  1.33s/it]

INFO 04-08 15:08:51 [model_runner.py:1598] Graph capturing finished in 36 secs, took 0.24 GiB
INFO 04-08 15:08:51 [llm_engine.py:448] init engine (profile, create kv cache, warmup model) took 45.19 seconds



Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


### Data Prep
<a name="Data"></a>

We're using OpenAI's famous GSM8K dataset!

In [8]:
from google.colab import files
uploaded = files.upload()

Saving Alpaca_FinalDataSet.json to Alpaca_FinalDataSet (1).json


In [9]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="Alpaca_FinalDataSet.json", split="train")
dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 99
})

Let's look at the first row:

In [10]:
dataset[0]["instruction"]

'Why is deduplication important when processing package scans?'

In [11]:
dataset[0]["output"]

'To prevent counting the same package multiple times due to duplicate scan or task events.'

We now create a system prompt which can be customized. We add 4 extra symbols for working out or thinking / reasoning sections and a final answer:

In [9]:
def create_system_prompt(instruction, input_text, output, eos_token="</s>"):
    """Creates a system prompt in Alpaca format with optional input"""

    if input_text.strip():
        system_prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_text}

### Output:
{output}{eos_token}"""
    else:
        system_prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Output:
{output}{eos_token}"""

    return system_prompt


Let's map the dataset! and see the first row:

In [10]:
dataset = dataset.map(lambda row: {
    "text": create_system_prompt(row["instruction"], row["input"], row["output"])
})

dataset[0]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

{'instruction': 'Why is deduplication important when processing package scans?',
 'input': 'Deduplication\nSome single scan events at a facility may generate several Task core events, which will result in appearance of the same package in different task core data feeds. Similarly, the same package may go through several different scans during sort at a facility, appearing in different task core domain events. This may lead to the issue of counting the same package several times. For that reason, Deduplication rule should be applied when counting Volume:\nEvery unique package (=package with unique Tracking ID) should be counted only once per Sort type per Operational Date per Facility.\nIf package is processed in multiple sorts of the same type (e.g. same package appears in “OB” and in “OB1” sorts), it should be counted only once, as the sort type that happened first.\nIf package is processed in multiple sorts of different types (e.g. same package appears in “OB” and in “IB” sorts), it 

In [12]:
def build_grpo_prompt(example):
    instruction = example["instruction"]
    input_text = example.get("input", "").strip()
    output = example["output"].strip()

    if input_text:
        prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
        ### Instruction:
        {instruction}

        ### Input:
        {input_text}

        ### Output:
        {output}"""
    else:
        prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

        ### Instruction:
        {instruction}

        ### Output:
        {output}"""

    return {
        "prompt": prompt,
        "completion": output
    }


In [13]:
dataset_new = dataset.map(build_grpo_prompt)
dataset_new[0]

{'instruction': 'Why is deduplication important when processing package scans?',
 'input': 'Deduplication\nSome single scan events at a facility may generate several Task core events, which will result in appearance of the same package in different task core data feeds. Similarly, the same package may go through several different scans during sort at a facility, appearing in different task core domain events. This may lead to the issue of counting the same package several times. For that reason, Deduplication rule should be applied when counting Volume:\nEvery unique package (=package with unique Tracking ID) should be counted only once per Sort type per Operational Date per Facility.\nIf package is processed in multiple sorts of the same type (e.g. same package appears in “OB” and in “OB1” sorts), it should be counted only once, as the sort type that happened first.\nIf package is processed in multiple sorts of different types (e.g. same package appears in “OB” and in “IB” sorts), it 

In [14]:
# Just keep prompt + completion
dataset = dataset_new.map(lambda x: {
    "prompt": x["prompt"].strip(),
    "completion": x["completion"].strip()
}, remove_columns=[col for col in dataset.column_names if col not in ["prompt", "completion"]])
dataset[0]

{'prompt': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n        ### Instruction:\n        Why is deduplication important when processing package scans?\n\n        ### Input:\n        Deduplication\nSome single scan events at a facility may generate several Task core events, which will result in appearance of the same package in different task core data feeds. Similarly, the same package may go through several different scans during sort at a facility, appearing in different task core domain events. This may lead to the issue of counting the same package several times. For that reason, Deduplication rule should be applied when counting Volume:\nEvery unique package (=package with unique Tracking ID) should be counted only once per Sort type per Operational Date per Facility.\nIf package is processed in multiple sorts of the same type (e.g. same package appears in “OB” and i

Get the maximum prompt length so we don't accidentally truncate it!

In [15]:
from transformers import AutoTokenizer

# Tokenize to get length
def get_prompt_length(example):
    tokens = tokenizer(example["prompt"], return_tensors="pt", truncation=False)
    return {"prompt_length": len(tokens["input_ids"][0])}

In [16]:
dataset_with_lengths = dataset.map(get_prompt_length)
max_length = max(dataset_with_lengths["prompt_length"])
print(f"📏 Max prompt length (tokens): {max_length}")

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

📏 Max prompt length (tokens): 772


In [24]:
dataset_with_lengths = dataset_new.map(get_prompt_length)
max_length = max(dataset_with_lengths["prompt_length"])
print(f"📏 Max prompt length (tokens): {max_length}")

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

📏 Max prompt length (tokens): 769


In [23]:
max_seq_length = 2048
filtered_dataset = dataset_with_lengths.filter(lambda ex: ex["prompt_length"] <= max_seq_length)


Filter:   0%|          | 0/99 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [17]:
max_prompt_length = 772 + 1 # + 1 just in case!

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 1000,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [18]:
def reward_similarity(prompts, completions):
    # You can use simple keyword heuristics here
    # Iterate through each completion in the list
    for completion in completions:
        if "error" in completion.lower():
            return 0.1  # Return 0.1 if "error" is found in any completion
    return 1.0  # Return 1.0 if "error" is not found in any completion

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    args = training_args,
    train_dataset = dataset,
    reward_funcs = [reward_similarity]
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 99 | Num Epochs = 42 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 97,255,424/3,310,005,248 (2.94% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / reward_similarity


<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [1]:
from unsloth import FastLanguageModel
import torch

# # Load the original (pre-trained, non-GRPO) model
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/llama-3-3b-instruct-bnb-4bit",  # or 8b if your GPU allows
#     max_seq_length = 128,
#     load_in_4bit = True,
#     fast_inference = True
# )

model.eval()

prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Why is deduplication important when processing package scans?

### Output:"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-08 15:05:45 [__init__.py:239] Automatically detected platform cuda.


NameError: name 'model' is not defined

In [ ]:
text = tokenizer.apply_chat_template([
    {"role": "user", "content": "What is the sqrt of 101?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.79it/s, est. speed input: 211.51 toks/s, output: 67.29 toks/s]


'The square root of 101 is approximately 10.05.'

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Verify LoRA is actually trained!

In [ ]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_saved_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

Now we load the LoRA and test:

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.21s/it, est. speed input: 39.47 toks/s, output: 77.58 toks/s]


"<start_working_out>\nTo find the square root of 101, we can use a calculator or estimate it manually. Since 101 is not a perfect square, we'll have to use a calculator for an accurate result.\n\nUsing a calculator, the square root of 101 is approximately 10.05.\n\nAlternatively, we can estimate it manually by looking for the closest perfect squares. The closest perfect squares are 100 and 121. Since 101 falls between them, its square root will be between 10 and 11.\n\n10^2 = 100\n11^2 = 121\n\n101 is closer to 121, so the square root of 101 will be slightly less than 11.\n\n10.05 is the estimated square root of 101.\n\n<end_working_out>\n<SOLUTION>10.05</SOLUTION>"

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
